# <u><h><center> Pauta Taller 1a</center></h></u>
## IIC2115 - Programación Como Herramienta para la Ingeniería
### Ayudante: Felipe Fuentes

In [17]:
import random #para cuando algo sea aleatorio
from abc import abstractmethod, ABC #para metodos abstractos

## <u>Clases</u>

Definimos la clase abstracta:

In [18]:
class Individuo(ABC): #abstracta
    
    def __init__(self, mesa):
        self.edad= random.randint(18, 85)
        self.n_mesa= random.randint(1, mesa)
        self.tiempo= 0 
        self.tiempo_votacion()
    
    @abstractmethod
    def tiempo_votacion(self):
        pass
    

Y las subclases que heredan de esta:

In [19]:
class Votante(Individuo): #herencia
    
    def tiempo_votacion(self): 
        self.tiempo= int(self.edad*0.05)
        
    def __str__(self):
        return f"votante de edad {self.edad} que vota en la mesa {self.n_mesa}" 

Creamos las clases fila

In [20]:
    
class Fila(): 
    
    def __init__(self):
        self.votantes=[] 
    
    def tipo(self): #definimos sus metodos
        pass
    
    def agregar_votante(self):
        pass


In [21]:
class FilaInterna(Fila):
    
    #polimorfismo
    
    def tipo(self):
        return "fila local"
    
    def agregar_votante(self, votante):
        self.votantes.append(votante) #agregación
        print(f"[FILA IN] Un votante ingreso a la fila interna en la mesa {votante.n_mesa}")
    
class FilaExterna(Fila):
    
    #polimorfismo
    
    def tipo(self):
        return "fila exterior"
    
    def agregar_votante(self, votante):
        self.votantes.append(votante) #agregación
        print(f"[FILA EX] Un votante ingreso a la fila externa en la mesa {votante.n_mesa}")

In [22]:
class Mesa():
    
    def __init__(self, id):
        self._id= id
        self.ocupado= None
        self.fila_interna= FilaInterna() #composición
        self.fila_externa= FilaExterna() #composición
        
    def ingresar_local(self):
        if len(self.fila_externa.votantes):
            votante= self.fila_externa.votantes.pop(0)
            self.fila_interna.agregar_votante(votante)
            return True
        return False

In [23]:
class Local(): 
     
    def __init__(self, N, aforo):
        self.n_mesas= N
        self.mesas= {}
        self.tiempo_espera= {}
        self.aforo= aforo
        self._gen_mesas()
        
    def _gen_mesas(self):
        for id in range(1, self.n_mesas+1):
            self.mesas[id]= Mesa(id)
            self.tiempo_espera[id]= [] 
    
    def revisar_aforo(self):
        contador_aforo= 0
        for mesa in self.mesas:
            contador_aforo+= len(self.mesas[mesa].fila_interna.votantes)
            if self.mesas[mesa].ocupado:
                contador_aforo +=1
        print(f"[AFORO] El aforo del local es de {self.aforo} y la cantidad de votantes dentro del local es de {contador_aforo}")
        if contador_aforo < self.aforo:
            return True
        return False
    
    def tick(self,pos):
        if self.mesas[pos].ocupado != None:
            self.mesas[pos].ocupado.tiempo -= 1
            if self.mesas[pos].ocupado.tiempo <= 0:
                print('[Mesa] termina atencion de {}'.format(self.mesas[pos].ocupado))
                self.mesas[pos].ocupado = None
    
    def mover_filas(self):
        votantes_afuera= True
        while votantes_afuera:
            votantes_afuera= False
            for mesa in self.mesas:
                if self.revisar_aforo():
                    if self.mesas[mesa].ingresar_local():
                        votantes_afuera= True
                else:
                    votantes_afuera= False
                    break
    
    def ingresar_votante(self, votante):  #agregamos el votante al final de la fila externa de su mesa
        self.mesas[votante.n_mesa].fila_externa.agregar_votante(votante)

Simular

In [24]:
N= 3
local= Local(N, 5)
max_t= 80
for t in range(max_t):
    votante = None if random.random() >= 0.9 else Votante(N)
    if votante!= None:
        print('[Llegada]  En tiempo de simulacion t={} min. Llego un {} con un tiempo promedio de {}'.format(t, votante, votante.tiempo))
        local.ingresar_votante(votante) #agregamos el votante nuevo.


     #revisamos si podemos mover votantes de las filas exteriores a dentro del local.
    local.mover_filas()

    for mesa in local.mesas: 
        if local.mesas[mesa].ocupado== None and len(local.mesas[mesa].fila_interna.votantes):
            local.mesas[mesa].ocupado= local.mesas[mesa].fila_interna.votantes.pop(0)
            local.tiempo_espera[mesa].append(local.mesas[mesa].ocupado.tiempo)
        local.tick(mesa)
print("\n")
print('Estadísticas:')
for mesa in local.mesas: 
    if len(local.tiempo_espera[mesa])!=0:
        tiempo_promedio = sum(local.tiempo_espera[mesa]) / len(local.tiempo_espera[mesa])
    else:
        tiempo_promedio = "No es calculable"
    tiempo_total = sum(local.tiempo_espera[mesa])
    print('Tiempo promedio de espera mesa {} es de {} min.'.format(mesa ,tiempo_promedio))
    print('Tiempo total de atención de la mesa {} fue de {} min'.format(mesa, sum(local.tiempo_espera[mesa])))
    print('Total de votantes atendidos por la mesa {}: {}'.format(mesa,len(local.tiempo_espera[mesa])))

[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 0
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 0
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 0
[Llegada]  En tiempo de simulacion t=1 min. Llego un votante de edad 21 que vota en la mesa 1 con un tiempo promedio de 1
[FILA EX] Un votante ingreso a la fila externa en la mesa 1
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 0
[FILA IN] Un votante ingreso a la fila interna en la mesa 1
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 1
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 1
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 1
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro del local es de 1
[AFORO] El aforo del local es de 5 y la cantidad de votantes dentro de